In [ ]:
import pandas as pd
import io
import ast
import json

test_csv = io.StringIO("""filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
                        WSCT4429.JPG,3923430,"{}",0,0,"{}","{}"
                        WSCT4430.JPG,3943204,"{}",0,0,"{}","{}"
                        WSCT4445.JPG,3935018,"{}",1,0,"{""name"":""rect"",""x"":2169,""y"":438,""width"":118,""height"":126}","{""Insect"":{""Carabid"":true}}"
                        WSCT4431.JPG,3933753,"{}",0,0,"{}","{}"
                        WSCT4445.JPG,3935018,"{}",1,0,"{""name"":""rect"",""x"":2169,""y"":438,""width"":118,""height"":126}","{""Insect"":{""Staphylinidae"":true}}"
                        
                    """)


def extract_categories(csv_path):
    df = pd.read_csv(csv_path)  # Load CSV

    unique_categories = set()  # Store unique category names

    for attributes in df['region_attributes']:
        try:
            attr_dict = json.loads(attributes)  # Convert string to dictionary
            if "Insect" in attr_dict:  # Check if 'Insect' key exists
                unique_categories.update(attr_dict["Insect"].keys())  # Extract category names
        except json.JSONDecodeError:
            continue  # Skip if JSON is malformed

    # Create category dictionary with unique, sorted category names
    categories = [
        {"id": idx + 1, "name": cat, "supercategory": "Insect"}
        for idx, cat in enumerate(sorted(unique_categories))  # Sorted to ensure consistency
    ]

    return categories

categories_list = extract_categories(test_csv)

# Print extracted COCO categories
print(json.dumps(categories_list, indent=4))



df = pd.read_csv(test_csv)
df['image_id'] = pd.Categorical(df['filename'], ordered=True).codes #df['filename'].astype('category').cat.codes # create a unique file id for each unique filename-row
df['image_id'] = "boop" + df['image_id'].astype(str)
df['category_id']= pd.Categorical(df['region_attributes'],ordered= True).codes
df["annid"] = df.index

print(df.to_string())

for row in df.itertuples():
    #print(type(row.region_attributes))
    jsonobj = json.loads(row.region_attributes)
    print(jsonobj)
    if row.region_count <= 0:
        continue
    print(list(jsonobj["Insect"].keys())[0])
    print(json.loads(row.region_shape_attributes))
    shape = json.loads(row.region_shape_attributes)
    bbox = [shape["x"], shape["y"], shape["width"], shape["height"]]
    print(bbox)

                               filename  file_size file_attributes  region_count  region_id                                    region_shape_attributes                  region_attributes image_id  category_id  annid
0                          WSCT4429.JPG    3923430              {}             0          0                                                         {}                                 {}    boop0            2      0
1                          WSCT4430.JPG    3943204              {}             0          0                                                         {}                                 {}    boop1            2      1
2                          WSCT4445.JPG    3935018              {}             1          0  {"name":"rect","x":2169,"y":438,"width":118,"height":126}        {"Insect":{"Carabid":true}}    boop3            0      2
3                          WSCT4431.JPG    3933753              {}             0          0                                                 